## Introducing Distributed Word Vectors

튜토리얼 part 2는 Word2Vec 알고리즘에 의해 만들어지는 분포적 단어 벡터를 사용하는 데 집중합니다. 
part 2와 part 3에서는 part 1에서보다 파이썬에 더 친숙할 것이라 가정합니다. 

Word2Vec은, 2013년 구글에 의해 출판되었으며, 단어의 분포적 표상을 학습하는 신경망 구현입니다. 다른 신경망 구조들도 단어의 표상들을 학습해 왔지만, 주요한 문제는 모델을 훈련하는 데 많은 시간이 걸린다는 점이었습니다. Word2Vec은 다른 모델에 비하여 빠르게 학습을 수행합니다. 

Word2Vec은 의미 있는 표상을 만들어내기 위해 라벨을 필요로 하지 않습니다. 이는 유용한데, 실제 세상에 있는 대부분의 데이터가 라벨이 붙어있지 않기 때문입니다. 만약 네트워크가 충분한 훈련 데이터를 받는다면(수백억 단어), Word2Vec은 아주 흥미로운 특성을 가진 단어 벡터들을 산출합니다. 유사한 의미를 가진 단어들이 군집을 이루어 나타나고, 군집들은 서로 떨어져서 어떤 의미관계들을 나타냅니다. 벡터 수학을 사용하여 유추(analogy)와 같은 것이 재현될 수 있습니다. 가장 유명한 예는 "king - man + woman = queen." 입니다. 

분포적 단어 벡터들은 강력하고 단여 예측과 번역 등 많은 곳에서 적용 가능합니다. 여기서는 감성 분석에 적용을 시도할 것입니다.  

## Using word2vec in Python

파이썬에서는 `gensim` 패키지에서 구현된 훌륭한 word2vec을 사용할 것입니다(gemsim 설치 필요).  

비록 Word2vec이 다른 딥러닝 알고리즘처럼 GPU를 요구하지는 않지만, 구글 버전이든 파이썬 버전이든 multi-threading(컴퓨터에서 여러 프로세스를 병렬적으로 처리하여 시간을 단축)에 의존합니다. 따라서 합리적인 시간 안에 모델을 훈련시키기 위해 cython을 설치할 필요가 있습니다. cython이 설지되지 않는다면, 몇 분이 아니라 며칠이 걸릴 것입니다.    

## Preparing to Train a Model

이제부터 핵심으로 들어갑니다. 먼저 part 1에서와 마찬가지로 판다스에서 데이터를 읽어들일 것입니다. 이번에는 50,000개의 라벨 없는 추가 리뷰인 `unlabeledTrain.tsv`도 사용합니다. Bag of words와는 달리, Word2Vec은 라벨이 없는 데이터로부터 학습하기 때문입니다.  

In [ ]:
import pandas as pd

train = pd.read_csv( "labeledTrainData.tsv", header=0, delimiter = "\t", quoting=3)
test =  pd.read_csv( "testData.tsv", header=0, delimiter = "\t", quoting=3)
unlabeled_train = pd.read_csv ("unlabeledTrainData.tsv", header=0, delimiter = "\t", quoting=3)

In [ ]:
#데이터의 크기를 확인합니다. 
print("Read %d labeled train review, %d labeled test reviews, and %d unlabeld reviews\n" 
      % (train["review"].size, test["review"].size, unlabeled_train["review"].size))

데이터를 정제하기 위해 사용하는 함수는 part 1과 비슷하지만 몇 가지 다른 점이 있습니다. 우선, Word2Vec을 훈련하기 위해서는 stopword를 제거하지 않는 편이 낫습니다. 왜냐하면 고품질의 단어 벡터를 산출하기 위해 알고리즘이 문장의 광범위한 맥락에 의존하기 때문입니다. 이러한 이유로, 우리는 아래의 함수에서 stopword 제거를 선택적인 것으로 둘 것입니다. 마찬가지로 숫자 역시 지우지 않는 편이 좋지만, 여기서는 제거합니다.   

In [ ]:
# cleansing을 위한 모듈을 불러오기
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist (review, remove_stopwords=False):
    # 문서를 단어 연쇄로 바꾸는 함수
    # 선택적으로 stopword 제거
    # 단어의 리스트를 반환
    # 1. HTML 제거하기
    review_text = BeautifulSoup(review).get_text()
    # 2. 단어가 아닌 것들을 제거하기
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # 3. 모두 소문자로 바꾸기, 단어 단위로 잘라서 리스트에 넣기
    words = review_text.lower().split()
    # 4. 선택적으로 stop words 제거하기
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    # 단어의 리스트를 반환하기
    return(words)

Word2Vec의 입력 형식은 리스트의 리스트입니다. (리스트1 : 문장, 리스트2: 단어)
* [[sophie가,Word2Vec을,연습하고,있다], [Word2Vec은, 리스트의, 리스트를, 요구한다]]

문단을 문장으로 나누는 것은 간단하지 않으며, 많은 요령들이 있다(마침표를 기준으로 자른다든지). 여기서는 NLTK의 punkt tokenizer for sentence split을 사용한다.

In [ ]:
# 문장 분할을 위해 punkt tokenizer 다운로드
import nltk.data

# punkt tokenizer 불러오기
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# 리뷰를 문장 단위로 분할하기 위한 함수 정의
def review_to_sentences (review, tokenizer, remove_stopwords=False) : 
    # 리뷰를 문장 단위로 분할한다. 문장 단위 리스트를 반환한다. 각 문장은 리스트로 이루어져 있다. 
    # 1. NLTK tokenizer를 사용하여 문단을 문장 단위로 변환한다. 
    raw_sentences = tokenizer.tokenize(review.strip())
    # 2. 각 문장에 대해 반복문을 작성한다. 
    sentences = []
    for raw_sentence in raw_sentences:
        # 만약 문장이 비어있다면 스킵
        if len(raw_sentence) > 0:
            #그렇지 않으면 review_to_wordlist를 불러서 단어들의 리스트를 만들고 리스트에 추가
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords=False))
    # 문장의 리스트를 반환함. 즉 리스트의 리스트를 반환함.
    return (sentences)
    

이제 Word2Vec 입력을 위한 데이터 처리 함수가 준비되었습니다. 

In [ ]:
sentences = [] # 문장의 빈 리스트를 초기화

print ("Parcing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)
print ("Parcing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


URL에 관한 몇 가지 warning 메시지가 있겠지만 URL을 고려할 생각이 아니라면 걱정하지 않아도 됩니다. 
이제 part 1과 어떻게 다른지 살펴보기로 합니다.

In [ ]:
print(len(sentences)) # 약 850,000개가 나와야 한다고 합니다. 

In [ ]:
print (sentences[1])

## Training and Saving Your Model

이제 모델을 훈련할 준비가 되었습니다. 실행 속도와 모델의 품질에 영향을 미치는 많은 파라미터들이 있습니다. 아래를 참고하시기 바랍니다. 

* **Architecture**
    - Architecture 옵션은 skip-gram(기본값) 또는 continuous bag of words가 있습니다. skip-gram은 약간 느리지만 더 나은 결과를 산출합니다. 
    
    
* **Training algorithm**
    - Hierarchical softmax(기본값) 또는 negative sampling
    - 여기서는 기본값이 잘 작동합니다. 
    
    
* **Downsampling of frequent words**
    - 구글의 문서는 .0001과 .001 사이의 값을 추천합니다. 
    - 이 과제에서는 0.001에 가까울수록 최종 모델의 정확도를 향상시킵니다.
    
    
* **Word vector demensionality**
    - feature의 수가 많을수록 런타임이 길어지지만, 보통은(항상 그런 것은 아니고) 더 나은 모델을 만들어 줍니다. 
    - 합리적인 값은 10~300 사이입니다. 여기서는 300을 사용합니다. 
    
    
* **Context/window size**
    - 훈련 알고리즘이 고려할 맥락의 단어 수입니다.
    - 10이 hierachical softmax에서 잘 작동하는 것 같습니다. 
    
    
* **Worker threads**
    - 실행을 위한 병렬 처리 숫자입니다. 컴퓨터에 따라 다르지만 대부분의 시스템에서 4에서 6 사이가 작동합니다.
    
    
* **Minimum word count**
    - 이 파라미터는 의미 있는 단어의 크기를 제한하는 데 도움을 줍니다. 합리적인 값은 10에서 100 사이입니다. 
    - 이 과제에서는 영화 제목이 30번씩 나오기 때문에, 최솟값을 40으로 설정하여 개별 영화 제목에 너무 많은 중요성이 부여되지 않도록 합니다. 전체 어휘 크기는 15,000개입니다. 

파라미터를 선택하기는 쉽지 않지만, 우리가 파라미터를 한 번 선택하면 Word2Vec을 모델링하는 것은 매우 간단합니다.

In [ ]:
import logging
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 파라미터 설정
num_features = 300 # 단어 벡터의 차원 수
min_word_count = 40 # 단어의 최소 빈도 
num_workers = 4  # 병렬 처리할 thread의 수
context = 10 # 맥락의 크기
downsampling = 1e-3 # 자주 나타나는 단어에 대한 Downsample 설정

# 모델 초기화 와 훈련  (시간이 조금 걸립니다)
from gensim.models import word2vec
print ("Training model...")
model =  word2vec.Word2Vec(sentences, workers = num_workers,
                          size=num_features, 
                          min_count = min_word_count,
                          window = context,
                          sample = downsampling)

# 만약 모델을 이후에 훈련시키지 않을 거라면 메모리 효율성을 위해 아래의 코드를 실행할 것
model.init_sims(replace = True)

# 모델 이름을 파라미터를 넣어 지어 두면 좋습니다. 
model_name = "300features_40minwords_10context"
model.save(model_name)

## Exploring the Model Results

`model.wv.doesnt_match`함수는 집합에 있는 단어 중 다른 것들과 가장 유사하지 않은 단어를 추정합니다.  

In [ ]:
model.wv.doesnt_match("man woman child kitchen".split())

In [ ]:
model.wv.doesnt_match("france england germany berlin".split())

In [ ]:
model.wv.doesnt_match("paris berlin london austria".split()) #완벽하지는 않음.

`wv.most_similar'함수를 사용하면 모델이 단어를 어떻게 군집화했는지 추측할 수 있습니다. 

In [ ]:
model.wv.most_similar("man")

In [ ]:
model.wv.most_similar("queen")

In [ ]:
model.wv.most_similar("awful")

## Numeric Representations of Words

In [ ]:
# part2에서 만든 모델을 불러온다.

from gensim.models import Word2Vec
model =  Word2Vec.load("300features_40minwords_10context")

In [ ]:
type(model)

In [ ]:
model.wv.vectors.shape

각 단어의 vector는 다음의 방식으로 접근할 수 있습니다. 

In [ ]:
model.wv["flower"]

In [ ]:
model.index2word

## From Words to Paragraphs, Attempt 1 : Vector Averaging

IMDB 데이터셋이 가진 한 가지 어려운 점은 리뷰마다 길이가 다르다는 점입니다. 따라서,모든 리뷰에 대해 같은 길이를 가지도록 개별 단어 백터를 변환할 방법을 찾을 필요가 있습니다. 

각 단어가 300차원의 벡터이기 때문에, 우리는 각 리뷰에서 단어들을 조합하기 위한 벡터 연산을 할 수 있습니다. 우리가 시도할 한 가지 방법은 단순하게 리뷰의 단어 벡터들의 평균값을 구하는 것입니다(stopwords가 노이즈를 발생시킬 수 있기 때문에 이러한 목적으로 제거하였습니다.)  

In [ ]:
import numpy as np


def makeFeatureVec(words, model, num_features):
    #문단의 모든 단어 벡터의 평균을 구하는 함수
    #empty numpy array 초기화 
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    #
    #Index2word 는 모델의 어휘에 있는 단어들의 이름을 포함. 이들을 속도 향상을 위해 집합으로 변경
    index2word_set = set(model.wv.index2word)
    #
    #리뷰의 각 단어에 대해 반복, 만약 모델의 어휘집에 있다면 합계에 feature vector를 포함함. 
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec =np.add(featureVec,model.wv[word])
    # 평균값을 구하기 위해 결과를 단어의 수로 나눔
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    # 주어진 리뷰 집합들(각각은 단어의 리스트로 되어 있음)에 대하여 각각의 평균을 구하고 2D numpy array로 반환
    # counter를 초기화 
    counter = 0
    # 2D numpy array를 사전 할당(속도 향상을 위해)
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 반복
    for review in reviews:
        #1000개 리뷰를 처리할 때마다 메시지 출력
        if (counter%1000 == 0):
            print ("Review %d of %d" % (counter, len(reviews)))
        # 위에서 정의한 함수를 불러옴 
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        #  counter 숫자를 하나씩 증가시키기
        counter = counter+1
    return reviewFeatureVecs

In [ ]:
#*************************************************************
# 훈련 셋과 검증 셋의 feature vector의 평균을 구함
# 위에서 정의한 함수들을 사용함. 
# num_features = 300
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist (review, remove_stopwords = True))
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)
print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test ["review"]:
    clean_test_reviews.append( review_to_wordlist(review,
                                                 remove_stopwords = True))
testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features)

이제 문단의 평균 벡터를 사용해서 랜텀포레스트를 훈련합니다. 
part 1에서처럼 훈련 데이터와 검증 데이터를 사용합니다. 

In [ ]:
#훈련 데이터를 랜덤 포레스트 모형에 적합, 트리 개수 =100개 
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier (n_estimators = 100)

print("Fitting a random forest to labeled training data...")
forest = forest.fit ( trainDataVecs, train["sentiment"])

# 검증 및 결과 추출
result =  forest.predict(testDataVecs)

# 검증 결과 쓰기
output = pd.DataFrame( data = {"id": test["id"], "sentiment" : result})
output.to_csv ( "Word2Vec_AverageVectors.csv", index = False, quoting=3)

우리 Bag of words 모델보다 약간 더 낮은 결과 점수를 확인할 수 있습니다. 

벡터의 평균값을 넣는 방식이 좋은 결과를 내지 못했는데, 그렇다면 더 좋은 방법이 있을까요? 단어 벡터에 가중치를 두는 표준적인 방법 중 하나는 "tf-idf" 가중치입니다. "tf-idf"는 문서 전체에서 어떤 단어의 중요도를 측정합니다. 파이썬에서는 sklearn의 `TfidfVectorizer`를 사용하는데, 이는 우리가 part 1에서 사용한 `CountVectorizer`와 유사한 인터페이스를 가집니다. 그러나 이 방식으로 단어 벡터에 가중치를 반영하면, 성능이 향상됩니다.

## From Words to Paragraphs, Attempt 2: Clustering 

Word2Vec은 의미적인 관련어의 클러스터를 만들어 줍니다. 그러므로 가능한 또 다른 접근은 같은 군집 안에 속한 단어들을 이용하는 것입니다. 벡터를 이러한 방식으로 묶는 것을 "vector quantization"이라고 합니다. 이를 위해 우선 단어 클러스터의 center를 찾아야 합니다. 즉, K-means와 같은 클러스터링 알고리즘을 사용합니다. 

K-means에서, 우리는 K를 설정해야 합니다. K는 클러스터의 수입니다. 어떻게 우리가 클러스터의 개수를 결정할 수 있을까요? 몇 번의 시도에 따르면 5개 정도의 단어를 하나의 클러스터로 하는 것이 많은 단어를 포함하는 클러스터보다 더 좋은 결과를 보였습니다. 클러스터링 코드는 아래에 있습니다. 

"K"의 개수가 많은 K-means는 실행에 많은 시간이 걸립니다. 아래의 코드는 튜토리얼 작성자에 따르면 40분 이상이 걸립니다(저는 약 20분이 걸렸습니다). 

In [ ]:
from sklearn.cluster import KMeans
import time

start = time.time() #현재 시각을 구하는 함수(시작 시각)

# "k"의 개수를 어휘 집합의 1/5로 설정. 
word_vectors = model.wv.vectors
num_clusters = int(model.wv.vectors.shape[0]/5)

# k-means object를 초기화하고 이를 centroid 추출에 사용
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict (word_vectors)

#끝나는 시각을 구하고, 얼마나 시간이 걸렸는지 출력
end = time.time()

elapsed = end - start

print ("Time taken for K Means clustering: ", elapsed, "seconds.")


각 단어에 대한 클러스터 할당은 이제 `idx`에 저장되며, 우리의 Word2Vec 모델로부터 얻은 어휘집은 여전히 `model.wv.index2word`에 있습니다. 편의를 위해 이들을 하나의 디렉토리에 모아 둡니다.  

In [ ]:
# word/index 딕셔너리, 어휘집의 단어를 클러스터 숫자에 연결
word_centroid_map = dict(zip(model.wv.index2word, idx))

조금 추상적이지만, 클러스터에 들어 있는 것들을 살펴봅시다. 클러스터는 달라질 수 있는데, 이는 Word2Vec이 Random number seed에 의존하기 때문입니다. 아래는 클러스터 0부터 9까지의 단어를 출력한 것입니다. 

In [ ]:
# 10 개의 클러스터
for cluster in range(0,10):
    # 클러스터 숫자를 출력
    print("\nCluster %d" % cluster)
    #해당 클러스터에 속한 단어들을 찾아서 출력
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if(list(word_centroid_map.values())[i] == cluster):
            words.append(list(word_centroid_map.keys())[i])
    print(words)

클러스터마다 결과의 품질이 다르지만, 어쨌든 각 단어를 클러스터(또는 centroid)에 할당하였습니다. 
이번에는 리뷰들을 bags of centroids로 변환하는 함수를 작성합니다. 이 작업은 Bag of words와 같지만 개별 단어 대신 의미적인 관련 클러스터를 사용합니다. 

In [ ]:
def create_bag_of_centroids ( wordlist, word_centroid_map):
    # 클러스터 숫자가 단어/클러스터의 가장 높은 클러스터 인덱스와 같다
    num_centroids = max(word_centroid_map.values())+1
    # bag of centroid vector를 사전 할당한다. 
    bag_of_centroids = np.zeros( num_centroids, dtype="float32")
    # 각 리뷰에 속한 단어들에 대하여 반복한다. 
    #만약 단어가 어휘집 안에 있다면 어떤 클러스터에 속했는지 찾고, 해당 클러스터 빈도에 1을 더한다.
    #예를 들어 5번 클러스터에 속한 단어가 1개였는데, 
    #반복문을 돌리는 과정에서 또 출현한다면 5번 클러스터에 속한 단어 개수를 2개로 바꾸어 주라는 것. 
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] +=1
    # "bag of centroids"를 반환한다. 
    return bag_of_centroids

In [ ]:
# 훈련 셋의 bag of centroids의 array를 사전 할당한다. 
train_centroids = np.zeros((train["review"].size, num_clusters),dtype="float32")
# 훈련 셋의 review를 bags of centroids로 변환한다.
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids ( review, word_centroid_map)
    counter += 1
test_centroids = np.zeros((test["review"].size, num_clusters), dtype = "float32")
counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids (review, word_centroid_map)
    counter +=1

In [ ]:
# 랜덤 포레스트 모형 적합하고, 예측 
forest =  RandomForestClassifier (n_estimators = 100)

#랜텀 포레스트 적합 (몇 분 걸림)
print ("Fitting the random forest to labeled training data...")
forest = forest.fit(train_centroids, train["sentiment"])
result = forest.predict(test_centroids)

# 테스트 결과 쓰기
output = pd.DataFrame(data={"id":test["id"],"sentiment":result})
output.to_csv("BagOfCentroids.csv", index=False, quoting=3)

결과 점수를 보면 Bag of Words 모델을 사용했을 때보다 약간 낮거나 같음을 알 수 있습니다. 